In [0]:
#!pip install tensorflow-gpu==2.0.0beta

In [0]:
from tensorflow.keras.layers import Lambda, Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.losses import mse, binary_crossentropy
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K
import numpy as np
import matplotlib.pyplot as plt
import argparse
import os

In [11]:
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
os.chdir(r'/content/gdrive/My Drive/Colab Notebooks/Autoencoders/')

In [0]:
# z = z_mean + sqrt(var) * epsilon

def sampling(args):

    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean = 0 and std = 1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

In [14]:

# CIFAR10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

image_size = x_train.shape[1]
original_dim = image_size * image_size
x_train = np.reshape(x_train, [-1, original_dim])
x_test = np.reshape(x_test, [-1, original_dim])
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# network parameters
input_shape = (original_dim, )
intermediate_dim = 512
batch_size = 128
latent_dim = 2
epochs = 50

# VAE model = encoder + decoder
# build encoder model
inputs = Input(shape=input_shape, name='encoder_input')
x = Dense(intermediate_dim, activation='relu')(inputs)
z_mean = Dense(latent_dim, name='z_mean')(x)
z_log_var = Dense(latent_dim, name='z_log_var')(x)

# use reparameterization trick to push the sampling out as input
# note that "output_shape" isn't necessary with the TensorFlow backend
z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

# instantiate encoder model
encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')
encoder.summary()
plot_model(encoder, to_file='vae_mlp_encoder.png', show_shapes=True)

# build decoder model
latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
x = Dense(intermediate_dim, activation='relu')(latent_inputs)
outputs = Dense(original_dim, activation='sigmoid')(x)

# instantiate decoder model
decoder = Model(latent_inputs, outputs, name='decoder')
decoder.summary()
plot_model(decoder, to_file='vae_mlp_decoder.png', show_shapes=True)

# instantiate VAE model
outputs = decoder(encoder(inputs)[2])
vae = Model(inputs, outputs, name='vae_mlp')

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      [(None, 1024)]       0                                            
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 512)          524800      encoder_input[0][0]              
__________________________________________________________________________________________________
z_mean (Dense)                  (None, 2)            1026        dense_3[0][0]                    
__________________________________________________________________________________________________
z_log_var (Dense)               (None, 2)            1026        dense_3[0][0]                    
____________________________________________________________________________________________

In [15]:
models = (encoder, decoder)
data = (x_test, y_test)


reconstruction_loss = binary_crossentropy(inputs,
                                              outputs)

reconstruction_loss *= original_dim
kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = K.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)
vae.compile(optimizer='adam')
vae.summary()

W0711 19:35:42.456447 139832624809856 training_utils.py:1237] Output decoder missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to decoder.


Model: "vae_mlp"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      [(None, 1024)]       0                                            
__________________________________________________________________________________________________
encoder (Model)                 [(None, 2), (None, 2 526852      encoder_input[0][0]              
__________________________________________________________________________________________________
decoder (Model)                 (None, 1024)         526848      encoder[1][2]                    
__________________________________________________________________________________________________
tf_op_layer_clip_by_value_1/Min [(None, 1024)]       0           decoder[1][0]                    
____________________________________________________________________________________________

In [16]:
vae.fit(x_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(x_test, None))

Train on 150000 samples, validate on 30000 samples
Epoch 1/50
150000/150000 [==============================] - 44s 294us/sample - loss: 650.6047 - val_loss: 645.3723
Epoch 2/50
150000/150000 [==============================] - 43s 286us/sample - loss: 644.1156 - val_loss: 643.5928
Epoch 3/50
150000/150000 [==============================] - 43s 287us/sample - loss: 642.9342 - val_loss: 643.2314
Epoch 4/50
150000/150000 [==============================] - 43s 288us/sample - loss: 642.2630 - val_loss: 642.6723
Epoch 5/50
150000/150000 [==============================] - 43s 289us/sample - loss: 641.7485 - val_loss: 642.9198
Epoch 6/50
150000/150000 [==============================] - 43s 290us/sample - loss: 641.4007 - val_loss: 641.4827
Epoch 7/50
150000/150000 [==============================] - 44s 290us/sample - loss: 641.0230 - val_loss: 641.7365
Epoch 8/50
150000/150000 [==============================] - 43s 289us/sample - loss: 640.6578 - val_loss: 641.1716
Epoch 9/50
150000/150000 [===